# Feature Selection

In [ ]:
import time
import datetime
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')


from sklearn.model_selection import train_test_split

## Extracting  Features (Day of week / Day of Month)

In [ ]:
df = df.sort_values(['date'])

In [ ]:
# Extracting new features based on pickup date

df['date']=df['date'].astype('datetime64')

# Extract the pickup day of the month from the datetime column
df['PU_day_of_month'] = df['date'].dt.day.astype(np.uint8)

# Extract the pickup day of the week (0-6) from the datetime column
df['PU_day_of_week'] = df['date'].dt.weekday.astype(np.uint8)

df.head(5)

,date,PULocationID,count,PU_day_of_month,PU_day_of_week
0,2023-01-01,42,199.0,1,6
166,2023-01-01,76,107.0,1,6
167,2023-01-01,191,105.0,1,6
168,2023-01-01,217,103.0,1,6
169,2023-01-01,22,106.0,1,6


## Extracting  Features (last_day_deman_cnt / last_week_deman_cnt)


In [ ]:
df['last_day'] = ''
df['last_week'] = ''
for i in df.index:
    df['last_day'].loc[i] = df['date'].loc[i] - datetime.timedelta(days=1)
    df['last_week'].loc[i] = df['date'].loc[i] - datetime.timedelta(days=7)

In [ ]:
def last_demand_count(df,last_date,LID):
    return (df[(df['date'] == last_date) & (df['PULocationID'] == LID)]['count']).values[0]


df['last_day_demand_cnt'] = 0
df['last_week_demand_cnt'] = 0
for i in df.index:
    try:
        df['last_day_demand_cnt'].loc[i] = last_demand_count(df,df['last_day'].loc[i],df['PULocationID'].loc[i])
    except IndexError:
        df['last_day_demand_cnt'].loc[i] = 0

    try:
            df['last_week_demand_cnt'].loc[i] = last_demand_count(df,df['last_week'].loc[i],df['PULocationID'].loc[i])
    except IndexError:
        df['last_week_demand_cnt'].loc[i] = 0

df.head(5)

,date,PULocationID,count,PU_day_of_month,PU_day_of_week,last_day,last_week,last_day_demand_cnt,last_week_demand_cnt
0,2023-01-01,42,199.0,1,6,2022-12-31 00:00:00,2022-12-25 00:00:00,0,0
166,2023-01-01,76,107.0,1,6,2022-12-31 00:00:00,2022-12-25 00:00:00,0,0
167,2023-01-01,191,105.0,1,6,2022-12-31 00:00:00,2022-12-25 00:00:00,0,0
168,2023-01-01,217,103.0,1,6,2022-12-31 00:00:00,2022-12-25 00:00:00,0,0
169,2023-01-01,22,106.0,1,6,2022-12-31 00:00:00,2022-12-25 00:00:00,0,0


In [ ]:
df.to_parquet('../data/featureDS.parquet')

In [ ]:
test_data = df[df['date'] >= datetime.datetime(2023, 4, 1)]
df = df[df['date'] < datetime.datetime(2023, 4, 1)]

In [ ]:
df = df.drop(['date', 'last_day', 'last_week'], axis=1)
test_data = test_data.drop(['date', 'last_day', 'last_week'], axis=1)

In [ ]:
X = df.drop('count', axis=1)
target = df['count']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, shuffle=False)